In [44]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [45]:
# Загрузка данных
data = pd.read_excel("DynamicData_for_prediction.xlsx")


In [46]:
# Исключение столбца "YEARS"
data = data.drop("YEARS", axis=1)

In [60]:
# Извлечение координат из первой строки столбцов X и Y
coordinates = data.iloc[0][['X:I1', 'Y:I1', 'X:I2', 'Y:I2', 'X:I3', 'Y:I3', 
                            'X:P1', 'Y:P1', 'X:P2', 'Y:P2', 'X:P3', 'Y:P3']].values


In [61]:
# Создание матрицы смежности на основе расстояния между скважинами
def adjacency_matrix(coordinates):
    num_wells = len(coordinates)
    adj_matrix = np.zeros((num_wells, num_wells))

    for i in range(num_wells):
        for j in range(i+1, num_wells):
            distance = np.linalg.norm(coordinates[i] - coordinates[j])
            adj_matrix[i, j] = 1 / (distance + 1e-6)  # Добавляем небольшое значение, чтобы избежать деления на ноль
            adj_matrix[j, i] = 1 / (distance + 1e-6)  # Матрица симметрична

    return adj_matrix


In [ ]:
# Создание общей матрицы смежности
adj_matrix = adjacency_matrix(coordinates)

In [63]:
# Исключение данных для скважины P3 из обучающего набора
data_train = data.drop(['WBHP:P3', 'WLPR:P3', 'WWCT:P3', 'WOPR:P3', 'MD:P3', 'poro:P3', 'perm:P3', 'X:P3', 'Y:P3'], axis=1)


In [64]:
# Преобразование данных в массив numpy
all_features = data_train.values


In [65]:
# Стандартизация признаков
scaler = StandardScaler()
scaled_features = scaler.fit_transform(all_features)


In [54]:
# Разделение данных на признаки (features) и целевые переменные (targets)
features = scaled_features[:, :-1]  # Исключаем дебит нефти из признаков
targets = scaled_features[:, -1]    # Дебит нефти становится целевой переменной


In [55]:
# Конвертация в тензоры PyTorch
features_train_tensor = torch.tensor(features, dtype=torch.float32)
adj_matrix_tensor = torch.tensor(adj_matrix, dtype=torch.float32)
targets_train_tensor = torch.tensor(targets, dtype=torch.float32)


In [66]:
# Определение модели
class GraphNeuralNetwork(nn.Module):
    def __init__(self):
        super(GraphNeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(features_train_tensor.shape[1], 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)

    def forward(self, x, adj_matrix):
        x = torch.relu(self.fc1(x))
        x = torch.matmul(adj_matrix, x)
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [67]:
# Создание экземпляра модели
model = GraphNeuralNetwork()


In [68]:
# Определение функции потерь и оптимизатора
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [69]:
# Обучение модели
num_epochs = 100
losses = []
for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(features_train_tensor, adj_matrix_tensor)
    loss = criterion(output.squeeze(), targets_train_tensor)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())

RuntimeError: mat1 and mat2 shapes cannot be multiplied (12x12 and 90x64)